In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
googlestore = pd.read_csv('googleplaystore.csv')
polarity =  pd.read_csv('googleplaystore_user_reviews.csv')

In [3]:
#record 10472 has no 'Category' value. Proceed with removal
googlestore=googlestore.drop(10472)

# 1 Convert the app sizes to a number

In [4]:
googlestore.head(10)


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19M,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29M,"1,000,000+",Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1M,"10,000+",Free,0,Everyone,Art & Design;Creativity,"July 3, 2018",2.8,4.0.3 and up


In [5]:
def convert_to_bytes(i):
    if i[-1] == 'k':
        i = i[:-1] #remove last letter
        i = float(i)
        i = i*1024
        return i
    if i[-1] == 'M':
        i = i[:-1] #remove last letter
        i = float(i)
        i = i*1048576
        return i
    if i[-1] == 'G':
        i = i[:-1] #remove last letter
        i = float(i)
        i = i*1073741824
        return i
    else:
        return i

In [6]:
googlestore['Size_Bytes'] = googlestore['Size'].apply(convert_to_bytes)


In [7]:
googlestore.head(10)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Size_Bytes
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,1.99229e+07
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,1.46801e+07
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,9.12261e+06
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,2.62144e+07
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,2.93601e+06
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up,5.87203e+06
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19M,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up,1.99229e+07
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29M,"1,000,000+",Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up,3.04087e+07
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up,3.4603e+07
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1M,"10,000+",Free,0,Everyone,Art & Design;Creativity,"July 3, 2018",2.8,4.0.3 and up,3.25059e+06


# 2 Convert the number of installs to a number



In [8]:
googlestore['Installs'].head(10)

0        10,000+
1       500,000+
2     5,000,000+
3    50,000,000+
4       100,000+
5        50,000+
6        50,000+
7     1,000,000+
8     1,000,000+
9        10,000+
Name: Installs, dtype: object

In [9]:
#find any any character that is NOT a digit and delete it.
googlestore['Installs'] = googlestore['Installs'].str.replace("[^0-9]", "") 
googlestore['Installs'] = pd.to_numeric(googlestore['Installs'])

In [10]:
googlestore.head(10)#check


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Size_Bytes
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,10000,Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,1.99229e+07
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,1.46801e+07
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,9.12261e+06
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,50000000,Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,2.62144e+07
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,2.93601e+06
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,50000,Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up,5.87203e+06
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19M,50000,Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up,1.99229e+07
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29M,1000000,Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up,3.04087e+07
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,1000000,Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up,3.4603e+07
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1M,10000,Free,0,Everyone,Art & Design;Creativity,"July 3, 2018",2.8,4.0.3 and up,3.25059e+06


# 3 Transform “Varies with device” into a missing value


In [11]:
googlestore = googlestore.replace("Varies with device", np.nan, regex = True)


In [12]:
#checking in columns where that value was present
googlestore[googlestore['Size']=='Varies with device'] 
googlestore[googlestore['Current Ver']=='Varies with device']
googlestore[googlestore['Android Ver']=='Varies with device'] 

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Size_Bytes


# 4 Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 or 4.2)


In [13]:
def convert(x):
    if x is np.nan:
        return x #if x is nan return nan
    else: 
        match = re.search(r"([\d.]*\d+)", x) # match the dotted number 
        if match:
            return match.group() #return the dotted number

In [14]:
googlestore["Current Ver"] = googlestore["Current Ver"].apply(convert)
googlestore["Android Ver"] = googlestore["Android Ver"].apply(convert)

In [15]:
googlestore.sort_index(inplace = True)

googlestore[["Current Ver", "Android Ver"]].head(10) 

,Current Ver,Android Ver
0,1.0.0,4.0.3
1,2.0.0,4.0.3
2,1.2.4,4.0.3
3,NaN,4.2
4,1.1,4.4
5,1.0,2.3
6,1.1,4.0.3
7,6.1.61.1,4.2
8,2.9.2,3.0
9,2.8,4.0.3


In [16]:
if googlestore["Current Ver"].str.contains("Varies with device").any():
    print ("Column Android Ver contains varies with device") #check if actually there are no 'varies with device'

In [17]:
if googlestore["Android Ver"].str.contains("Varies with device").any():
    print ("Column Android Ver contains varies with device") #check if actually there are no 'varies with device'

# 5 Remove the duplicates


In [18]:
googlestore[googlestore["App"] == "8 Ball Pool"] #check how many times this app is present

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Size_Bytes
1675,8 Ball Pool,GAME,4.5,14198297,52M,100000000,Free,0,Everyone,Sports,"July 31, 2018",4.0.0,4.0.3,54525952.0
1703,8 Ball Pool,GAME,4.5,14198602,52M,100000000,Free,0,Everyone,Sports,"July 31, 2018",4.0.0,4.0.3,54525952.0
1755,8 Ball Pool,GAME,4.5,14200344,52M,100000000,Free,0,Everyone,Sports,"July 31, 2018",4.0.0,4.0.3,54525952.0
1844,8 Ball Pool,GAME,4.5,14200550,52M,100000000,Free,0,Everyone,Sports,"July 31, 2018",4.0.0,4.0.3,54525952.0
1871,8 Ball Pool,GAME,4.5,14201891,52M,100000000,Free,0,Everyone,Sports,"July 31, 2018",4.0.0,4.0.3,54525952.0
1970,8 Ball Pool,GAME,4.5,14201604,52M,100000000,Free,0,Everyone,Sports,"July 31, 2018",4.0.0,4.0.3,54525952.0
3953,8 Ball Pool,SPORTS,4.5,14184910,52M,100000000,Free,0,Everyone,Sports,"July 31, 2018",4.0.0,4.0.3,54525952.0


In [19]:
googlestore['Reviews'] = pd.to_numeric(googlestore['Reviews']) #need this way to get the max

In [20]:
googlestore = googlestore.loc[googlestore.groupby('App').Reviews.idxmax()] #the most reviewed is supposed to be the last one

In [21]:
googlestore[googlestore["App"] == "8 Ball Pool"] #check how many times this app is present


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Size_Bytes
1871,8 Ball Pool,GAME,4.5,14201891,52M,100000000,Free,0,Everyone,Sports,"July 31, 2018",4.0.0,4.0.3,54525952.0


# 6 For each category, compute the number of apps


In [22]:
googlestore[["App", "Category"]].groupby(['Category']).count()


,App
Category,
ART_AND_DESIGN,61
AUTO_AND_VEHICLES,85
BEAUTY,53
BOOKS_AND_REFERENCE,222
BUSINESS,420
COMICS,56
COMMUNICATION,315
DATING,170
EDUCATION,107


# 7 For each category, compute the average rating


In [23]:
googlestore[["Category", "Rating"]].groupby(['Category']).mean()

,Rating
Category,
ART_AND_DESIGN,4.359322
AUTO_AND_VEHICLES,4.190411
BEAUTY,4.278571
BOOKS_AND_REFERENCE,4.344970
BUSINESS,4.098479
COMICS,4.181481
COMMUNICATION,4.121484
DATING,3.980451
EDUCATION,4.354717


# 8 Create two dataframes: one for the genres and one bridging apps and genres. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity

In [24]:
genres = googlestore['Genres'] #database of genres

In [25]:
column = genres.str.split(";", n = 2, expand = True)
column #two columns with two genres

,0,1
8884,Sports,None
8532,Social,None
324,Comics,None
4541,Tools,None
4636,Communication,None
5940,News & Magazines,None
10252,Sports,None
1393,Health & Fitness,None
8483,Education,None
7738,Communication,None


In [26]:
g = pd.DataFrame() #dataframes for genres
g["Primo Genere"] = column[0]
g["Secondo Genere"] = column[1] #add them to a df

In [27]:
#dataframe of App
App = pd.DataFrame()
App = googlestore[["App"]]
type(App)

pandas.core.frame.DataFrame

In [28]:
bridging_table = pd.merge(App, g, left_index = True, right_index = True)


In [29]:
bridging_table['Primo Genere'].fillna(value=pd.np.nan, inplace=True)
bridging_table['Secondo Genere'].fillna(value=pd.np.nan, inplace=True)
bridging_table.head()

,App,Primo Genere,Secondo Genere
8884,"""i DT"" Fútbol. Todos Somos Técnicos.",Sports,NaN
8532,+Download 4 Instagram Twitter,Social,NaN
324,- Free Comics - Comic Apps,Comics,NaN
4541,.R,Tools,NaN
4636,/u/app,Communication,NaN


# 9 For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)

In [30]:
#add column app to df_genre

s = bridging_table.set_index('App')[['Primo Genere', 'Secondo Genere']]
df22 = pd.get_dummies(s, prefix_sep="", prefix="", dtype=bool).max(level=0)
df = App.join(df22, on='App').fillna(dict.fromkeys(df22.columns, False))
df

/home/corra/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/corra/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,App,Action,Adventure,Arcade,Art & Design,Auto & Vehicles,Beauty,Board,Books & Reference,Business,...,Trivia,Video Players & Editors,Weather,Word,Action & Adventure,Brain Games,Creativity,Education,Music & Video,Pretend Play
8884,"""i DT"" Fútbol. Todos Somos Técnicos.",False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8532,+Download 4 Instagram Twitter,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
324,- Free Comics - Comic Apps,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4541,.R,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4636,/u/app,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5940,058.ba,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10252,1. FC Köln App,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1393,10 Best Foods for You,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8483,10 Minutes a Day Times Tables,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7738,10 WPM Amateur ham radio CW Morse code trainer,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# 10 For each genre, compute the average rating. What is the genre with highest average?


In [31]:
googlestore2 = bridging_table.merge(googlestore, on = "App") #use the column created

In [32]:
googlestore2.groupby('Primo Genere', as_index = False, sort = False)[['Rating']].mean()

,Primo Genere,Rating
0,Sports,4.225185
1,Social,4.247291
2,Comics,4.181481
3,Tools,4.040278
4,Communication,4.121790
5,News & Magazines,4.121569
6,Health & Fitness,4.243496
7,Education,4.293574
8,Books & Reference,4.343275
9,Puzzle,4.370732


In [33]:
print("the max value corresponds to the genre " + googlestore2.groupby('Primo Genere')['Rating'].mean().argmax())

the max value corresponds to the genre Events


/home/corra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  """Entry point for launching an IPython kernel.


# 11 For each app, compute the approximate income, obtain as a product of number of installs and price


In [35]:
#without modifying googlestore dataframe
price = pd.DataFrame()
install = pd.DataFrame()
income = pd.DataFrame()
income = googlestore #want to show also 'App' name

install['Installs'] = googlestore['Installs'].astype(float) #same type of price
price['Price'] = googlestore['Price'].str.replace(r'$','').astype(float) #remove $ symbol

#multiply of many installs times the price
income['Income'] = price['Price']*install['Installs'] 

In [36]:
#show app name and related income from more remunerative to less
income[['App', 'Income']].sort_values('Income', ascending=False) 

,App,Income
2241,Minecraft,69900000.0
5351,I am rich,39999000.0
5356,I Am Rich Premium,19999500.0
4034,Hitman Sniper,9900000.0
7417,Grand Theft Auto: San Andreas,6990000.0
2883,Facetune - For Free,5990000.0
5578,Sleep as Android Unlock,5990000.0
8804,DraStic DS Emulator,4990000.0
4367,I'm Rich - Trump Edition,4000000.0
4362,💎 I'm rich,3999900.0


# 12 For each app, compute its minimum and maximum Sentiment_polarity


In [38]:
polarity.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [39]:
#aggregate, compute sum of the elements with the same value in column 'App' 
#and show min and max of the column 'sentiment_polarity'.
polarity_grouped = polarity.groupby('App')
polarity_grouped.agg({'App' : np.sum, 'Sentiment_Polarity'  : [np.min, np.max]}) 


App  \
                                                                                                  sum   
App                                                                                                     
10 Best Foods for You                               10 Best Foods for You10 Best Foods for You10 B...   
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室                    104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室104 找工作 - 找工作 ...   
11st                                                11st11st11st11st11st11st11st11st11st11st11st11...   
1800 Contacts - Lens Store                          1800 Contacts - Lens Store1800 Contacts - Lens...   
1LINE – One Line with One Touch                     1LINE – One Line with One Touch1LINE – One Lin...   
2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif    2018Emoji Keyboard 😂 Emoticons Lite -sticker&g...   
21-Day Meditation Experience                        21-Day Meditation Experience21-Day Meditation ...   
2Date Dating App, Love and matching                 2Date Dating App, Love and matching2Date Datin...   
2GIS: directory & navigator                         2GIS: directory & navigator2GIS: directory & n...   
2RedBeans                                           2RedBeans2RedBeans2RedBeans2RedBeans2RedBeans2...   
2ndLine - Second Phone Number                       2ndLine - Second Phone Number2ndLine - Second ...   
30 Day Fitness Challenge - Workout at Home          30 Day Fitness Challenge - Workout at Home30 D...   
365Scores - Live Scores                             365Scores - Live Scores365Scores - Live Scores...   
3D Blue Glass Water Keyboard Theme                  3D Blue Glass Water Keyboard Theme3D Blue Glas...   
3D Color Pixel by Number - Sandbox Art Coloring     3D Color Pixel by Number - Sandbox Art Colorin...   
3D Live Neon Weed Launcher                          3D Live Neon Weed Launcher3D Live Neon Weed La...   
4 in a Row                                          4 in a Row4 in a Row4 in a Row4 in a Row4 in a...   
4K Wallpapers and Ultra HD Backgrounds              4K Wallpapers and Ultra HD Backgrounds4K Wallp...   
591房屋交易-租屋、中古屋、新建案、實價登錄、別墅透天、公寓套房、捷運、買房賣房行情、房價房貸查詢  591房屋交易-租屋、中古屋、新建案、實價登錄、別墅透天、公寓套房、捷運、買房賣房行情、房價...   
591房屋交易-香港                                          591房屋交易-香港591房屋交易-香港591房屋交易-香港591房屋交易-香港591房屋交...   
7 Cups: Anxiety & Stress Chat                       7 Cups: Anxiety & Stress Chat7 Cups: Anxiety &...   
7 Day Food Journal Challenge                        7 Day Food Journal Challenge7 Day Food Journal...   
7 Minute Workout                                    7 Minute Workout7 Minute Workout7 Minute Worko...   
7 Weeks - Habit & Goal Tracker                      7 Weeks - Habit & Goal Tracker7 Weeks - Habit ...   
8 Ball Pool                                         8 Ball Pool8 Ball Pool8 Ball Pool8 Ball Pool8 ...   
850 Sports News Digest                              850 Sports News Digest850 Sports News Digest85...   
8fit Workouts & Meal Planner                        8fit Workouts & Meal Planner8fit Workouts & Me...   
95Live -SG#1 Live Streaming App                     95Live -SG#1 Live Streaming App95Live -SG#1 Li...   
A Call From Santa Claus!                            A Call From Santa Claus!A Call From Santa Clau...   
A Manual of Acupuncture                             A Manual of AcupunctureA Manual of Acupuncture...   
...                                                                                               ...   
Hole19: Golf GPS App, Rangefinder & Scorecard       Hole19: Golf GPS App, Rangefinder & ScorecardH...   
Home Decor Showpiece Art making: Medium Difficulty  Home Decor Showpiece Art making: Medium Diffic...   
Home Scouting® MLS Mobile                           Home Scouting® MLS MobileHome Scouting® MLS Mo...   
Home Security Camera WardenCam - reuse old phones   Home Security Camera WardenCam - reuse old pho...   
Home Street – Home Design Game                      Home Street – Home Design GameHome Street – Ho...   
Hom